In [13]:
%matplotlib inline

In [16]:
import json
import random
import numpy as np
import pandas as pd
import networkx as nx
import requests
import re

#https://docs.potterdb.com/apis/rest#endpoints

In [17]:
all_characters = []

for i in range(1, 411):
    url = "https://api.potterdb.com/v1/characters?"
    response = requests.get(url + "page[number={}]&page[size=10]".format(i))
    all_characters.append(response.json())

In [18]:
#all_characters

In [19]:
character_list = []
for page in all_characters:
    for character in page['data']:
        if character["attributes"]["family_members"] != None:
            character_list.append(character)

In [20]:
len(character_list)

1367

In [27]:
family_dict = {}
for character in character_list:
    family_dict[character["id"]] = {} 
    family_dict[character["id"]]["relatives"] = character['attributes']["family_members"]  

In [28]:
#family_dict['b8b5e21f-6ad8-49a2-b0aa-e9ca7d825ee5']

In [29]:
g = nx.Graph()

In [30]:
for character in character_list:
    g.add_node(character["id"], name=character['attributes']['name'])

In [31]:
print("Nodes:", len(g.nodes))

Nodes: 1364


In [37]:
#character_list[0]

In [35]:
for i, left_character in enumerate(character_list):
    for j, right_character in enumerate(character_list):
        if i != j:
            for family in left_character["attributes"]["family_members"]:
                if " " in right_character["attributes"]["name"]:
                    if any(right_character["attributes"]["name"] in name for name in left_character["attributes"]["family_members"]):
                        g.add_edge(left_character["id"], right_character["id"])
                else:
                    if right_character["attributes"]["name"] == family.split(" )", 1)[0]:
                        g.add_edge(left_character["id"], right_character["id"])

In [36]:
nx.write_graphml(g, "harrypotter.graphml")